In [13]:
import pandas as pd
import http.client
import json

In [6]:
conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "xxx"
    }

conn.request("GET", "/fixtures?league=71&season=2023", headers=headers)

res = conn.getresponse()
data = res.read()

In [29]:
# Decode the byte string to a UTF-8 string
data_str = data.decode('utf-8')

# Parse the JSON data into a Python dictionary
data_dict = json.loads(data_str)

# Assuming the 'response' key contains a list of fixtures
df_normalized = pd.json_normalize(data_dict['response'])


In [31]:
df_normalized.columns

Index(['fixture.id', 'fixture.referee', 'fixture.timezone', 'fixture.date',
       'fixture.timestamp', 'fixture.periods.first', 'fixture.periods.second',
       'fixture.venue.id', 'fixture.venue.name', 'fixture.venue.city',
       'fixture.status.long', 'fixture.status.short', 'fixture.status.elapsed',
       'league.id', 'league.name', 'league.country', 'league.logo',
       'league.flag', 'league.season', 'league.round', 'teams.home.id',
       'teams.home.name', 'teams.home.logo', 'teams.home.winner',
       'teams.away.id', 'teams.away.name', 'teams.away.logo',
       'teams.away.winner', 'goals.home', 'goals.away', 'score.halftime.home',
       'score.halftime.away', 'score.fulltime.home', 'score.fulltime.away',
       'score.extratime.home', 'score.extratime.away', 'score.penalty.home',
       'score.penalty.away'],
      dtype='object')

In [32]:
df_filtrado = df_normalized.drop(['fixture.referee', 'fixture.timezone', 'fixture.date',
       'fixture.timestamp', 'fixture.periods.first', 'fixture.periods.second',
       'fixture.venue.id', 'fixture.venue.name', 'fixture.venue.city',
       'fixture.status.long', 'fixture.status.short', 'fixture.status.elapsed','league.country', 'league.logo',
       'league.flag','teams.home.logo','teams.away.logo','score.halftime.away','score.halftime.home'], axis = 1)

In [33]:
df_filtrado

,fixture.id,league.id,league.name,league.season,league.round,teams.home.id,teams.home.name,teams.home.winner,teams.away.id,teams.away.name,teams.away.winner,goals.home,goals.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away
0,1005649,71,Serie A,2023,Regular Season - 1,127,Flamengo,True,147,Coritiba,False,3.0,0.0,3.0,0.0,None,None,None,None
1,1005650,71,Serie A,2023,Regular Season - 1,120,Botafogo,True,126,Sao Paulo,False,2.0,1.0,2.0,1.0,None,None,None,None
2,1005651,71,Serie A,2023,Regular Season - 1,121,Palmeiras,True,1193,Cuiaba,False,2.0,1.0,2.0,1.0,None,None,None,None
3,1005652,71,Serie A,2023,Regular Season - 1,131,Corinthians,True,135,Cruzeiro,False,2.0,1.0,2.0,1.0,None,None,None,None
4,1005653,71,Serie A,2023,Regular Season - 1,794,RB Bragantino,True,118,Bahia,False,2.0,1.0,2.0,1.0,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1006024,71,Serie A,2023,Regular Season - 38,135,Cruzeiro,None,121,Palmeiras,None,NaN,NaN,NaN,NaN,None,None,None,None
376,1006025,71,Serie A,2023,Regular Season - 38,119,Internacional,None,120,Botafogo,None,NaN,NaN,NaN,NaN,None,None,None,None
377,1006026,71,Serie A,2023,Regular Season - 38,147,Coritiba,None,131,Corinthians,None,NaN,NaN,NaN,NaN,None,None,None,None
378,1006027,71,Serie A,2023,Regular Season - 38,118,Bahia,None,1062,Atletico-MG,None,NaN,NaN,NaN,NaN,None,None,None,None


In [37]:
dataframes_list = []

for fixture in df_filtrado['fixture.id']:
    conn = http.client.HTTPSConnection("v3.football.api-sports.io")
    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': "xxxx"
        }

    conn.request("GET", f"/fixtures/players?fixture={fixture}", headers=headers)

    res = conn.getresponse()
    data = res.read()
    players_data = json.loads(data.decode("utf-8"))
    # Check if the 'response' key is in the dictionary
    if 'response' in players_data:
        # Normalize the 'response' data and append the DataFrame to the list
        df_players = pd.json_normalize(players_data['response'])
        dataframes_list.append(df_players)

# Concatenate all the DataFrames into one
df_all_players_stats = pd.concat(dataframes_list, ignore_index=True)

In [76]:
# Initialize an empty list to store DataFrames for each player's statistics
# Initialize an empty list to store DataFrames for each player's statistics
dataframes_list = []

# Loop over each row in the DataFrame
for index, row in df_filtrado.iterrows():
    # 'players' is a list of dictionaries, so we iterate over it
    for player_info in row['players']:
        # Normalize the 'statistics' data for the current player
        # Assuming 'statistics' is a key in the player_info dictionary and it's not None
        if 'statistics' in player_info and player_info['statistics'] is not None:
            statistics_df = pd.json_normalize(player_info['statistics'])
            # Add the player ID, player name, team name, and team ID
            statistics_df['player_id'] = player_info['player']['id']
            statistics_df['player_name'] = player_info['player']['name']
            # Here we extract team name and team ID from the row in df_filtrado
            statistics_df['team_name'] = row['team.name']  # Replace with the actual column name for team name in df_filtrado
            statistics_df['team_id'] = row['team.id']  # Replace with the actual column name for team ID in df_filtrado
            # Append the flattened DataFrame to the list
            dataframes_list.append(statistics_df)

# Concatenate all the DataFrames in the list into one final DataFrame
df_all_players_statistics = pd.concat(dataframes_list, ignore_index=True)

# Specify the column order to ensure the desired columns are first
column_order = ['player_id', 'player_name', 'team_name', 'team_id'] + [col for col in df_all_players_statistics.columns if col not in ['player_id', 'player_name', 'team_name', 'team_id']]
df_all_players_statistics = df_all_players_statistics[column_order]


In [77]:
df_all_players_statistics

,offsides,games.minutes,games.number,games.position,games.rating,games.captain,games.substitute,shots.total,shots.on,goals.total,...,cards.red,penalty.won,penalty.commited,penalty.scored,penalty.missed,penalty.saved,player_id,player_name,team_name,team_id
0,None,98,1,G,6.9,False,False,None,None,None,...,0,None,None,0,0,0,10112,Santos,Flamengo,127
1,None,98,43,D,6.6,False,False,None,None,None,...,0,None,None,0,0,None,349001,Wesley,Flamengo,127
2,None,98,15,D,7.3,False,False,1,None,None,...,0,None,None,0,0,None,10089,Fabrício Bruno,Flamengo,127
3,None,98,4,D,8.2,False,False,None,None,None,...,0,None,None,0,0,None,10124,Léo Pereira,Flamengo,127
4,None,98,6,D,7.5,False,False,2,2,1,...,0,None,None,0,0,None,1771,Ayrton Lucas,Flamengo,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2686,None,None,18,D,None,False,True,None,None,None,...,0,None,None,0,0,None,356539,Júlio César,America Mineiro,125
2687,None,None,23,M,None,False,True,None,None,None,...,0,None,None,0,0,None,6122,Javier Méndez,America Mineiro,125
2688,None,None,9,F,None,False,True,None,None,None,...,0,None,None,0,0,None,10437,Wellington Paulista,America Mineiro,125
2689,None,None,44,F,None,False,True,None,None,None,...,0,None,None,0,0,None,352869,Adyson,America Mineiro,125
